In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [2]:
from helpers import load_csv_data

seed = 666
DATA_TRAIN_PATH = 'data/train.csv'

np.random.seed(seed)
y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## prepare cost function and error function

In [3]:
def support_vector(y, X, w):
    return 1 - y * (X @ w)

def clipped_support_vector(y, X, w):
    return np.clip(support_vector(y, X, w), 0, np.inf)

In [4]:
def calculate_primal_loss(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_samples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_samples)
    w: shape = (num_features)
    """
    v = clipped_support_vector(y, X, w)
    return np.sum(v) + lambda_ / 2 * np.sum(w ** 2)

In [18]:
def accuracy(y1, y2):
    return np.mean(y1 == y2)

def prediction(X, w):
    return (X @ w > 0) * 2 - 1

def calculate_accuracy(y, X, w):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_samples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_samples)
    w: shape = (num_features)
    """
    predicted_y = prediction(X, w)
    return accuracy(predicted_y, y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [6]:
def calculate_stochastic_gradient(y, X, w, lambda_, n):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_samples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_samples)
    w: shape = (num_features)
    N: num_samples
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    def is_support(y_n, x_n, w):
        """a datapoint is support if max{} is not 0. """
        return y_n * x_n @ w < 1
    
    x_n, y_n = X[n], y[n]
    grad = - y_n * x_n.T if is_support(y_n, x_n, w) else np.zeros_like(x_n.T)
    grad = np.squeeze(grad) + lambda_ * w
    return grad

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [19]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 1000
    gamma = 0.001
    lambda_ = 0.1
    
    num_samples, num_features = X.shape
    sample_indices = range(num_samples)
    w = np.random.rand(num_features)
    
    for cur_iter in range(max_iter):
        # n = sample one data point uniformly at random data from x
        num_of_points = random.sample(sample_indices, num_samples)
        for n in num_of_points: 
            loss = calculate_primal_loss(y, X, w, lambda_)
            grad = calculate_stochastic_gradient(y, X, w, lambda_, n)
            w -= grad
        
        if cur_iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=cur_iter, l=loss))
    
    print("accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

start_time = datetime.now()
sgd_for_svm_demo(y, X)
end_time = datetime.now()

print('it takes {}'.format(end_time - start_time))

Current iteration=0, the loss=14334542791.998497
Current iteration=100, the loss=3953154456.069556
Current iteration=200, the loss=2180147541.075439
Current iteration=300, the loss=2711344034.114188
Current iteration=400, the loss=9539527074.038664
Current iteration=500, the loss=4456815208.6956625
Current iteration=600, the loss=13325508595.616325
Current iteration=700, the loss=2240276930.0331845
Current iteration=800, the loss=1106479326.3990116
Current iteration=900, the loss=7153258192.411452
accuracy = 0.6614
it takes 0:46:38.813805


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [11]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_samples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_samples)
    w: shape = (num_features)
    N: num_samples
    """        
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = alpha[n]
    
    g = (y_n * x_n.dot(w) - 1)

    if old_alpha_n == 0:
        g = min(g, 0)
    elif old_alpha_n == 1.0 / lambda_:
        g = max(g, 0)
    else:
        g = g
    if g != 0:
        alpha[n] = min(
            max(old_alpha_n - g / (x_n.T.dot(x_n)), 0.0),
            1.0 / lambda_)
    
        # compute the corresponding update on the primal vector w
        w += 1.0 * (alpha[n] - old_alpha_n) * y_n * x_n
    return w

In [12]:
def calculate_dual_loss(y, X, w, alpha, lambda_):
    """calculate the loss for dual problem."""
    return np.sum(alpha * support_vector(y, X, w)) + lambda_ / 2 * np.sum(w ** 2)

In [16]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 10000
    gamma = 0.001
    lambda_ = 0.1

    num_samples, num_features = X.shape
    sample_indices = range(num_samples)
    w = np.zeros(num_features)
    alpha = np.zeros(num_samples)
    
    for cur_iter in range(max_iter):
        # n = uniformly random data point from x
        num_of_points = random.sample(sample_indices, num_samples)

        for iteration in num_of_points:
            n = sample_indices[iteration]
            w = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if cur_iter % 1000 == 0:
            # primal objective
            primal_value = calculate_primal_loss(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_loss(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('cur_iter:%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    cur_iter, primal_value, dual_value, duality_gap))
    print("accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

start_time = datetime.now()
coordinate_descent_for_svm_demo(y, X)
end_time = datetime.now()

print('it takes {}'.format(end_time - start_time))

cur_iter:0, primal:7912.75995, dual:0.00482, gap:7912.75514
cur_iter:1000, primal:3370.34738, dual:4.06859, gap:3366.27879
cur_iter:2000, primal:4042.18045, dual:8.04565, gap:4034.13480
cur_iter:3000, primal:4858.45927, dual:12.00038, gap:4846.45889
cur_iter:4000, primal:3335.35133, dual:15.94030, gap:3319.41103
cur_iter:5000, primal:3431.59161, dual:19.88587, gap:3411.70575
cur_iter:6000, primal:3433.61252, dual:23.82310, gap:3409.78942
cur_iter:7000, primal:4168.09126, dual:27.76696, gap:4140.32430
cur_iter:8000, primal:3482.56702, dual:31.71424, gap:3450.85278
cur_iter:9000, primal:3214.92190, dual:35.66699, gap:3179.25491
accuracy = 0.7154
it takes 0:06:03.481119
